In [4]:
import random
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import math
import json

In [7]:
class firefighterProblem:
    def __init__(self,fire_file, firefighter_file=None, node_file=None):
        #the section can be modified
        self.T_number = 150 #時間長度
        self.T = list([i for i in range(self.T_number + 1)]) #時間list
        self.P = {1:3,2:4} #各個消防單位時間處理的燃料量
        self.mode = "not random"
        
        self.model = gp.Model("FIREFIGHTER")
        self.M = 10
        self.epsilon=1e-4
        self.A_p = gp.tuplelist()
        self.A_f = gp.tuplelist()
        self.tau = gp.tupledict()
        self.lamb = gp.tupledict()
        self.K = set() #K=消防員集合
        self.Q = {}
        self.b = {}
        self.H = {}
        self.process = {}
        self.A_f_NEIGHBOR = {} #A_f_NEIGHBOR=與點i相鄰的點
        self.A_f_NEIGHBOR_T = {} #A_f_NEIGHBOR_T=紀錄 t-hi-Lambda(i,j)>=0 且 與j點相鄰的i點
        self.x = {}
        self.w = {}
        self.u = {}
        self.u_bar = {}
        self.v = {}
        self.v_bar = {}
        self.NODE_POS = {}
        
#        self.read_from_excel(node_file)
        self.read_from_excel(fire_file)
#         self.read_from_excel(firefighter_file)
        
    def read_from_excel(self, fileName):
            df = pd.read_excel(fileName, sheet_name = None)
            self.N_number = df["coordinates"].index[-1]+1
            self.N = set([i for i in range(1, self.N_number+1)])
            print(self.N)
            for i in df["fire_route"].iloc:
                u = int(i['i'])
                v = int(i['j'])
                time = i['travel time']
                self.lamb[u,v] = time
                self.A_f.append((u,v))
            for i in self.N:
                self.A_p.append((i,i))
            for i in df["firefighter_route"].iloc:
                u = int(i['i'])
                v = int(i['j'])
                firefighterIndex = i['k']
                if firefighterIndex not in self.K:
                    self.K.add(int(firefighterIndex))
                time = i['travel time']
                self.tau[u,v,firefighterIndex] = time
                if (u,v) not in self.A_p:
                    self.A_p.append((u,v)) 
            for i in range(self.N_number):
                self.NODE_POS[i+1] = (int(df["coordinates"].iloc[i]['x']), int(df["coordinates"].iloc[i]['y']))
            if self.mode == 'random':
                for i in range(self.N_number):
                    self.Q[i+1] = random.randint(2,5)
                    self.b[i+1] = random.randint(20,30)
                    self.H[i+1] = random.randint(2,5)
            else:
                for i in range(self.N_number):
                    self.Q[i+1] = int(df["coordinates"].iloc[i]['quantity'])
                    self.b[i+1] = int(df["coordinates"].iloc[i]['value'])
                    self.H[i+1] = int(df["coordinates"].iloc[i]['burning time'])
            print(len(self.H))
            self.N_D = df['ff_source']['N_D'].tolist()
            self.P = {1: int(df['ff_source']['P'].iloc[0]), 2: int(df['ff_source']['P'].iloc[1])}
            self.N_F = set(df['fire_source']['N_F'].tolist())
            for i in self.N_D:
                if math.isnan(i):
                    self.N_D.remove(i)
            self.N_D = set(self.N_D)
        
        #         if "fire_route" in fileName:
#             fire_df = pd.read_excel(fileName)
#             df_num = len(fire_df.index)
#             for i in range(df_num):
#                 u = int(fire_df.iloc[i]['i'])
#                 v = int(fire_df.iloc[i]['j'])
#                 time = fire_df.iloc[i]['travel time']
#                 self.lamb[u,v] = time
#                 self.A_f.append((u,v))
#         elif "firefighter_route" in fileName:
#             firefighter_df = pd.read_excel(fileName)
#             df_num = len(firefighter_df.index)
#             for i in self.N:
#                 self.A_p.append((i,i))
#             for i in range(df_num):
#                 u = int(firefighter_df.iloc[i]['i'])
#                 v = int(firefighter_df.iloc[i]['j'])
#                 firefighterIndex = firefighter_df.iloc[i]['k']
#                 if firefighterIndex not in self.K:
#                     self.K.add(int(firefighterIndex))
#                 time = firefighter_df.iloc[i]['travel time']
#                 self.tau[u,v,firefighterIndex] = time
#                 #避免在多消防員時重複紀錄arc set
#                 if (u,v) not in self.A_p:
#                     self.A_p.append((u,v))  
#         elif "nodeInformation" in fileName:
#             nodeInfo1 = pd.read_excel(fileName, 'coordinates')
#             df_num = len(nodeInfo1.index)
#             self.N_number = df_num
#             self.N = set([i for i in range(1, self.N_number+1)])
            
#             for i in range(self.N_number):
#                 self.NODE_POS[i+1] = (int(nodeInfo1.iloc[i]['x']), int(nodeInfo1.iloc[i]['y']))
#             if self.mode == 'random':
#                 for i in range(self.N_number):
#                     self.Q[i+1] = random.randint(2,5)
#                     self.b[i+1] = random.randint(20,30)
#                     self.H[i+1] = random.randint(2,5)
#             else:
#                 for i in range(self.N_number):
#                     self.Q[i+1] = int(nodeInfo1.iloc[i]['quantity'])
#                     self.b[i+1] = int(nodeInfo1.iloc[i]['value'])
#                     self.H[i+1] = int(nodeInfo1.iloc[i]['burning time'])
            
#             nodeInfo2 = pd.read_excel(fileName, 'source')
#             self.N_D = set(nodeInfo2['N_D'].tolist())
#             self.N_F = set(nodeInfo2['N_F'].tolist())
    def initialize(self):
        for k in self.K:
            self.process[k]={}
            for i in self.N:
                if i in self.N_D:
                    self.process[k][i] = 0
                else:
                    self.process[k][i] = math.ceil(self.Q[i] * self.H[i] / self.P[k])
        for l in self.N - self.N_D:                          #定義A_f_NEIGHBOR
            connect = self.A_f.select('*',l)
            self.A_f_NEIGHBOR[l]=[]
            for temp in connect:
                self.A_f_NEIGHBOR[l].append(temp[0])
        for j in self.N - self.N_D - self.N_F:                         #定義A_f_NEIGHBOR_T
            for t in self.T:
                self.A_f_NEIGHBOR_T[j, t]=[]
                for i in self.A_f_NEIGHBOR[j]:
                    if t - self.H[i] - self.lamb[i, j]>=0:
                        self.A_f_NEIGHBOR_T[j, t].append(i)
        #定義x[i,j,k,t]
        for k in self.K:
            for t in self.T:
                for i in range(len(self.A_p)):
                    self.x[self.A_p[i][0], self.A_p[i][1], k, t] = self.model.addVar(vtype='B', name="x[%d,%d,%d,%d]" % (self.A_p[i][0], self.A_p[i][1], k, t))
        #定義w[i,k,t]
        for k in self.K:
            for t in self.T:
                for i in self.N:
                    self.w[i,k,t] = self.model.addVar(vtype='B',name="w[%d,%d,%d]" % (i, k, t))

        #定義u[i,t]
        self.u = self.model.addVars(self.N, self.T, vtype="B", name="u")

        #定義u_bar[i,k,t]
        self.u_bar = self.model.addVars(self.N, self.K, self.T, vtype="B", name="u_bar")

        #定義v[i,t]
        self.v = self.model.addVars(self.N, self.T, vtype="B", name="v")
        
        #定義v_bar[i,t]
        self.v_bar = self.model.addVars(self.N, self.T, vtype="B", name="v_bar")        
        
        self.model.update()
        
        #原點flow blance
        for k in self.K:                        
            self.model.addConstr(gp.quicksum(self.x[i,j,k,0] for i,j in self.A_p) <= 1)
    
        #限定從depot出發
        for O in self.N_D:
            connect = self.A_p.select(O,'*')
            for k in self.K:
                self.model.addConstr(gp.quicksum(self.x[i,j,k,0] for i, j in connect) == 1)
                
        #depot不會被保護
        #self.model.addConstrs((self.u_bar[i,k,t]==0 for i in self.N_D for k in self.K for t in range(self.T_number+1)))

        #flow balance
        for k in self.K:
            for t in range(1,self.T_number):
                for j in self.N: 
                    in_connect = self.A_p.select('*',j)
                    out_connect = self.A_p.select(j,'*')
                    temp = 0 #in-degree
                    temp += self.w[j, k, t - 1] # t-1在j idle
                    
                    if j in self.N_D: #j in depot set會有u_bar，只是都為0
                        temp += self.u_bar[j, k, t]
                    else: #j not in depot set, 若現在的t > process time，則有u_bar且為非0
                        if self.process[k][j] <= t:
                            temp += self.u_bar[j, k, t - self.process[k][j]]
                    for m, n in in_connect:
                        if m != n and self.tau[m, n, k] <= t: #若現在的t>travel time，則會有x
                            temp += self.x[m, n, k, t - self.tau[m,n,k]]
                    self.model.addConstr(temp == gp.quicksum(self.x[n, w, k, t] for n, w in out_connect), name="flow") #in-degree = out-degree
        
        #若在t from i to i(i not include depot), 一定會是在t時刻開始保護或idle
        self.model.addConstrs((self.u_bar[i, k, t] + self.w[i, k, t] == self.x[i, i, k, t] for i in self.N - self.N_D for k in self.K for t in range(self.T_number)))
        
        #若在t from s to s(s is in deopt set), 一定會在t時刻idle
        self.model.addConstrs((self.w[s, k, t] == self.x[s, s, k, t]) for s in self.N_D for k in self.K for t in range(self.T_number))
        
        #每個node在T內只會開始燒、開始保護、或未影響
        self.model.addConstrs(self.u[i, t] + self.u_bar.sum(i, '*', t) <= 1 for i in self.N for t in self.T)
        
        #開始燒與已經燒之間的關係
        self.model.addConstrs(self.v[i, t] + self.u[i, t] == self.v[i, t+1] for i in self.N for t in self.T[0:-1])
        
        #開始保護與已經保護之間的關係
        self.model.addConstrs(self.v_bar[i, t] + self.u_bar.sum(i, '*', t) == self.v_bar[i, t+1] for i in self.N for t in self.T[0:-1]) #constrain 9
        
        #deopt在T內不會開始保護
        self.model.addConstrs(self.u_bar.sum(s, '*', '*') == 0 for s in self.N_D)
        
        #depot在T內不會開始燒
        self.model.addConstrs(self.u.sum(s, '*') == 0 for s in self.N_D)
        
        #火焰的延燒
        for j in self.N - self.N_D - self.N_F:
            for t in range(self.T_number):
                if len(self.A_f_NEIGHBOR_T[j, t]) == 0:
                    self.model.addConstr(self.u[j, t] == 0, name='test')
                else:
                    self.model.addConstr(gp.quicksum(self.u[i, t - self.H[i] - self.lamb[i, j]] for i in self.A_f_NEIGHBOR_T[j, t]) / self.M <= self.u[j, t] + self.v[j, t] + self.v_bar[j, t + 1])
                    self.model.addConstr(gp.quicksum(self.u[i, t - self.H[i] - self.lamb[i, j]] for i in self.A_f_NEIGHBOR_T[j, t]) >= self.u[j, t])
        
        #給定起火點
        for i in self.N_F:                           
            self.model.addConstr(self.u[i, 0] == 1)
        
        #給定所有節點已經燒的起始狀態
        for i in self.N:                           
            self.model.addConstr(self.v[i, 0] == 0)
    
        #給定所有節點已經保護的起始狀態
        for i in self.N:                            
            self.model.addConstr(self.v_bar[i, 0] == 0)

        #消防員不能去已經被燃燒的節點
        for k in self.K:                             
            for t in self.T:
                for l in range(len(self.A_p)):
                    if self.A_p[l][1] not in self.N_D:
                        if self.A_p[l][0] ==  self.A_p[l][1]:
                            if t + 2 <= self.T_number:
                                self.model.addConstr(self.M * (1 - self.v[self.A_p[l][1], t + 1]) >= self.x[self.A_p[l][0], self.A_p[l][1], k, t])
                            else:
                                self.model.addConstr(self.M * (1 - self.v[self.A_p[l][1], self.T_number]) >= self.x[self.A_p[l][0], self.A_p[l][1], k, t])
                        elif t + self.tau[self.A_p[l][0], self.A_p[l][1], k] + 1 <= self.T_number:
                            self.model.addConstr(self.M * (1 - self.v[self.A_p[l][1], t + self.tau[self.A_p[l][0], self.A_p[l][1], k]]) >= self.x[self.A_p[l][0], self.A_p[l][1], k, t])
                        else:
                            self.model.addConstr(self.M * (1 - self.v[self.A_p[l][1], self.T_number]) >= self.x[self.A_p[l][0], self.A_p[l][1], k, t])
        
        #設定目標式
        self.model.setObjective(gp.quicksum(gp.quicksum(self.u[i, t] for t in self.T) * self.b[i] for i in self.N - self.N_D) + 
                           gp.quicksum(self.epsilon * self.x[i, j, k, t] for (i, j, k, t) in self.x if i != j) +
                           gp.quicksum(self.epsilon * t * self.u_bar[i, k, t] for i in self.N - self.N_D for k in self.K for t in self.T), GRB.MINIMIZE)
        return self.model
        
    def showTextSol(self):
        print("x:")
        for k in self.K:
            print()
            print("消防員%d的路徑" % k)
            temp = [elem for elem in self.x if elem[2] == k]
            for (i, j, k, t) in temp:
                if self.x[i, j, k, t].X > self.epsilon:
                    if i != j:
                        print("在時刻 %d 從node%d 移動到 node%d" % (t, i, j), " ,travel time:", self.tau[i, j, k])
                    else:            
                        if self.u_bar[i,k,t].X == 1:
                            print("在時刻 %d 對node%d進行保護" % (t,i), " ,processing time:", self.process[k][i])
                        else:
                            print("在時刻 %d 在node%d idle" % (t,i))

#         print("w:")
#         for (i, k, t) in self.w:
#             if self.w[i, k, t].X > self.epsilon:
#                 print("w[%d,%d,%d]" % (i, k, t) , self.w[i, k, t].X)

#         print("u:")
#         for (i, t) in self.u:
#             if self.u[i,t].X > self.epsilon:
#                 print("u[%d,%d]" % (i,t), self.u[i,t].X)

#         print("u_bar:")
#         for (i, k, t) in self.u_bar:
#             if self.u_bar[i, k, t].X > self.epsilon:
#                 print("u_bar[%d,%d,%d]" % (i, k, t), self.u_bar[i, k, t].X)

#         print("v:")
#         for (i, t) in self.v:
#             if self.v[i, t].X > self.epsilon:
#                 print("v[%d,%d]" % (i,t), self.v[i, t].X)

#         print("v_bar:")
#         for (i, t) in self.v_bar:
#             if self.v_bar[i, t].X > self.epsilon:
#                 print("v_bar[%d,%d]" % (i,t), self.v_bar[i, t].X)
    def writeJson(self, file):
        data = {}
        data['NODE_POS'] = self.NODE_POS
        data['N'] = list(self.N)
        data['N_D'] = list(self.N_D)
        data['N_F'] = list(self.N_F)
        data['K'] = list(self.K)
        data['A_p'] = list([str(i) for i in self.A_p])
        data['A_f'] = list([str(i) for i in self.A_f])
        data['tau'] = dict((str(i), self.tau[i]) for i in self.tau)
        data['lamb'] = dict((str(i), self.lamb[i]) for i in self.lamb)
        data['T'] = self.T
        data['q'] = self.Q
        data['b'] = self.b
        data['p'] = self.P
        data['h'] = self.H
        
        temp = {}
        for (i, j, k, t) in self.x:
            temp[str((i, j, k, t))] = self.x[i, j, k, t].X
        data['x'] = temp

        temp = {}
        for (i, k, t) in self.w:
            temp[str((i, k, t))] = self.w[i, k, t].X
        data['w'] = temp

        temp = {}
        for (i, t) in self.u:
            temp[str((i, t))] = self.u[i, t].X
        data['u'] = temp

        temp = {}
        for (i, k, t) in self.u_bar:
            temp[str((i, k, t))] = self.u_bar[i, k, t].X
        data['u_bar'] = temp

        temp = {}
        for (i, t) in self.v:
            temp[str((i, t))] = self.v[i, t].X
        data['v']  = temp

        temp = {}
        for (i, t) in self.v_bar:
            temp[str((i, t))] = self.v_bar[i, t].X
        data['v_bar'] = temp
        
        json_data = json.dumps(data)
        with open(file[:-5]+"_data.json", "w") as file:
            file.write(json.dumps(data))

In [8]:
import os
files = os.listdir("./")

file_list = [f for f in files if os.path.isfile(os.path.join("./", f))and f[-5:] == ".xlsx"]

for file in file_list:
    print(file)

    ff = firefighterProblem(fire_file=file)
    model = ff.initialize()

    #run model and write lp file
    model.optimize()
    #model.write('test.lp')
    print("optimal value : ", model.ObjVal)

    ff.showTextSol()
    ff.writeJson(file)

FFP_n20_no1.xlsx
{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20}
20
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 50483 rows, 46508 columns and 180464 nonzeros
Model fingerprint: 0xdd3694b0
Variable types: 0 continuous, 46508 integer (46508 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+01]
  Objective range  [1e-04, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 34392 rows and 26038 columns
Presolve time: 1.61s
Presolved: 16091 rows, 20470 columns, 65063 nonzeros
Variable types: 0 continuous, 20470 integer (20470 binary)
Found heuristic solution: objective 175.0046000

Root relaxation: objective 2.668607e+01, 1096 iterations, 0.07 seconds (0.04 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/No

FFP_n20_no2.xlsx
{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20}
20
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 49922 rows, 46508 columns and 178717 nonzeros
Model fingerprint: 0x424b3055
Variable types: 0 continuous, 46508 integer (46508 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+01]
  Objective range  [1e-04, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 31588 rows and 24276 columns
Presolve time: 1.87s
Presolved: 18334 rows, 22232 columns, 73391 nonzeros
Variable types: 0 continuous, 22232 integer (22232 binary)
Found heuristic solution: objective 150.0145000

Root relaxation: objective 4.502457e+01, 4140 iterations, 0.43 seconds (0.29 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/No

     0     0   80.01329    0  354   80.02160   80.01329  0.01%     -    3s
     0     0   80.01336    0  910   80.02160   80.01336  0.01%     -    3s
H    0     0                      80.0200000   80.01336  0.01%     -    3s
     0     0   80.01336    0  916   80.02000   80.01336  0.01%     -    3s

Cutting planes:
  Gomory: 8
  MIR: 2
  StrongCG: 1
  Inf proof: 1
  Zero half: 31
  Mod-K: 1
  RLT: 2

Explored 1 nodes (2347 simplex iterations) in 3.82 seconds (1.62 work units)
Thread count was 8 (of 8 available processors)

Solution count 8: 80.02 80.0216 80.0218 ... 170.024

Optimal solution found (tolerance 1.00e-04)
Best objective 8.002000000000e+01, best bound 8.001335733534e+01, gap 0.0083%
optimal value :  80.02000000000001
x:

消防員1的路徑
在時刻 0 從node5 移動到 node16  ,travel time: 16.0
在時刻 16 從node16 移動到 node8  ,travel time: 6.0
在時刻 22 對node8進行保護  ,processing time: 5
在時刻 27 從node8 移動到 node16  ,travel time: 6.0
在時刻 33 從node16 移動到 node2  ,travel time: 8.0
在時刻 41 對node2進行保護  ,processing tim

   323   278   65.01944  112  327   65.01990   53.61056  17.5%   175   25s
H  331   278                      65.0198000   53.61056  17.5%   172   25s
H  334   233                      65.0186000   53.61056  17.5%   172   25s
H  445   355                      65.0185000   55.53910  14.6%   143   27s
H  605   463                      65.0184000   58.61154  9.85%   123   29s
   645   461 infeasible   36        65.01840   58.61154  9.85%   125   30s
H  738   505                      65.0183000   60.02737  7.68%   125   31s
*  782   497              41      65.0177000   60.02778  7.67%   127   32s
   865   619   60.03499   39  338   65.01770   60.02836  7.67%   133   35s
H  886   614                      65.0173000   60.02836  7.67%   137   35s
  1081   615   65.01539   11 1009   65.01730   60.02836  7.67%   126   40s
  1086   618   60.03180    8  865   65.01730   60.02836  7.67%   126   45s
  1090   621   60.03832   73  974   65.01730   60.02836  7.67%   125   50s
  1093   623   60.03365  

     0     0   62.52423    0  724  100.02030   62.52423  37.5%     -    2s
     0     0   62.52475    0  724  100.02030   62.52475  37.5%     -    2s
     0     0   62.52479    0  641  100.02030   62.52479  37.5%     -    2s
     0     0   62.52523    0  741  100.02030   62.52523  37.5%     -    2s
     0     0   62.52523    0  822  100.02030   62.52523  37.5%     -    3s
     0     0   62.52536    0  661  100.02030   62.52536  37.5%     -    3s
     0     0   62.52537    0  664  100.02030   62.52537  37.5%     -    3s
     0     0   62.52545    0  757  100.02030   62.52545  37.5%     -    3s
     0     0   62.52545    0  775  100.02030   62.52545  37.5%     -    3s
     0     0   62.52564    0  752  100.02030   62.52564  37.5%     -    3s
     0     0   62.52564    0  749  100.02030   62.52564  37.5%     -    3s
     0     2   62.52771    0  749  100.02030   62.52771  37.5%     -    3s
*   11    14               4      75.0398000   65.01922  13.4%   555    4s
    29    28   65.02020  

在時刻 127 在node13 idle
在時刻 128 在node13 idle
在時刻 129 在node13 idle
在時刻 130 在node13 idle
在時刻 131 在node13 idle
在時刻 132 在node13 idle
在時刻 133 在node13 idle
在時刻 134 在node13 idle
在時刻 135 在node13 idle
在時刻 136 在node13 idle
在時刻 137 在node13 idle
在時刻 138 在node13 idle
在時刻 139 在node13 idle
在時刻 140 在node13 idle
在時刻 141 在node13 idle
在時刻 142 在node13 idle
在時刻 143 在node13 idle
在時刻 144 在node13 idle
在時刻 145 在node13 idle
在時刻 146 在node13 idle
在時刻 147 在node13 idle
在時刻 148 在node13 idle
在時刻 149 在node13 idle

消防員2的路徑
在時刻 0 從node8 移動到 node4  ,travel time: 21.0
在時刻 21 對node4進行保護  ,processing time: 2
在時刻 23 從node4 移動到 node2  ,travel time: 10.0
在時刻 33 從node2 移動到 node1  ,travel time: 5.0
在時刻 38 對node1進行保護  ,processing time: 13
在時刻 51 在node1 idle
在時刻 52 在node1 idle
在時刻 53 在node1 idle
在時刻 54 在node1 idle
在時刻 55 在node1 idle
在時刻 56 在node1 idle
在時刻 57 在node1 idle
在時刻 58 在node1 idle
在時刻 59 在node1 idle
在時刻 60 在node1 idle
在時刻 61 在node1 idle
在時刻 62 在node1 idle
在時刻 63 在node1 idle
在時刻 64 在node1 idle
在時刻 65 在node1 idle
在時刻 66 在node1 

FFP_n20_no8.xlsx
{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20}
20
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 49012 rows, 46508 columns and 176140 nonzeros
Model fingerprint: 0x0fc9ad61
Variable types: 0 continuous, 46508 integer (46508 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+01]
  Objective range  [1e-04, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 37705 rows and 29041 columns
Presolve time: 1.34s
Presolved: 11307 rows, 17467 columns, 49097 nonzeros
Variable types: 0 continuous, 17467 integer (17467 binary)
Found heuristic solution: objective 155.0047000

Root relaxation: objective 7.502612e+01, 3666 iterations, 0.26 seconds (0.26 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/No

     0     0   78.99426    0  825  180.01080   78.99426  56.1%     -    4s
     0     0   79.00789    0  824  180.01080   79.00789  56.1%     -    4s
     0     0   79.05431    0  841  180.01080   79.05431  56.1%     -    4s
     0     0   79.14557    0  931  180.01080   79.14557  56.0%     -    4s
     0     0   79.14879    0  931  180.01080   79.14879  56.0%     -    4s
     0     0   79.15387    0 1011  180.01080   79.15387  56.0%     -    4s
H    0     0                     155.0244000   79.15387  48.9%     -    5s
H    0     0                      95.0327000   79.15387  16.7%     -    5s
     0     0   79.43504    0 1079   95.03270   79.43504  16.4%     -    5s
H    0     0                      95.0326000   79.43504  16.4%     -    5s
     0     0   79.43967    0 1080   95.03260   79.43967  16.4%     -    5s
     0     0   79.43969    0 1087   95.03260   79.43969  16.4%     -    5s
     0     0   79.77022    0  831   95.03260   79.77022  16.1%     -    6s
     0     0   79.78871  

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\$FFP_n20_no1.xlsx'

In [ ]:
ff = firefighterProblem(fire_file="FFP_n20_no1.xlsx")
model = ff.initialize()

#run model and write lp file
model.optimize()
#model.write('test.lp')
print("optimal value : ", model.ObjVal)

ff.showTextSol()
ff.writeJson(file)